# Data Collection

dataset link: http://mirlab.org/dataset/public/MIR-QBSH.zip


google forms link: https://forms.gle/z2QS9fVMf39mG6sk8

In [6]:
import os
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from midi2audio import FluidSynth

In [2]:
# Storing the path of the files in a list

In [17]:
# Directories containing the original and hummed audio files
original_dir = 'MIR-QBSH/MIR-QBSH/miditowav'
hummed_root_dir = 'MIR-QBSH/MIR-QBSH/waveFile'

# Collect all original files (assumes one original per song)
original_files = [os.path.join(original_dir, f) for f in os.listdir(original_dir) if f.endswith('.wav')]

# Prepare lists to store DataFrame columns
song_names = []
original_paths = []
hummed_paths = []

# Loop through each original file and find corresponding hummed files
for original_path in original_files:
    song_name = os.path.splitext(os.path.basename(original_path))[0]  # Extract song name (without extension)

    # Find corresponding hummed files in hummed_root_dir by traversing subdirectories
    for root, _, files in os.walk(hummed_root_dir):
        for file in files:
            if file.endswith('.wav') and song_name in file:  # Match song name
                hummed_path = os.path.join(root, file)
                
                # Append data for each hummed version
                song_names.append(song_name)
                original_paths.append(original_path)
                hummed_paths.append(hummed_path)

# Create the DataFrame
data = {
    'Song Name': song_names,
    'Original Audio Path': original_paths,
    'Hummed Audio Path': hummed_paths
}
audio_df = pd.DataFrame(data)

print(audio_df.head())  # Display the first few rows to verify


  Song Name         Original Spectrogram Path  \
0     00001  spectrograms/miditowav/00001.png   
1     00001  spectrograms/miditowav/00001.png   
2     00001  spectrograms/miditowav/00001.png   
3     00001  spectrograms/miditowav/00001.png   
4     00001  spectrograms/miditowav/00001.png   

                             Hummed Spectrogram Path  
0  spectrograms/hummed/year2006a/person00001/0000...  
1  spectrograms/hummed/year2006a/person00002/0000...  
2  spectrograms/hummed/year2006a/person00003/0000...  
3  spectrograms/hummed/year2006a/person00004/0000...  
4  spectrograms/hummed/year2006a/person00005/0000...  


In [4]:
audio_df.to_csv('audio_data.csv', index=False)

from midi2audio import FluidSynth
import os

soundfont_path = "Nice-Steinway-Lite-v3.0.sf2"

path = "MIR-QBSH/MIR-QBSH/midiFile"
savepath = "MIR-QBSH/MIR-QBSH/miditowav"

def converter(midipath,filename):

    fs = FluidSynth(soundfont_path)
    print(os.path.join(savepath,filename))
    fs.midi_to_audio(midipath, filename)

folder = []
for entry in os.listdir(path):
    if not os.path.isfile(entry):
        folder.append(entry)
print(folder)

for midi in folder:
    midipath = f"{path}\{midi}"
    filename = midi.replace('.mid', '.wav')
    midisavepath=f"{savepath}\{filename}"
    converter(midipath,midisavepath)

In [4]:
# Spectrogram generation function
def audio_to_spectrogram(audio_path, output_path, top_n=200):
    # Load the audio file
    y, sr = librosa.load(audio_path, sr=None)

    # Compute the Short-Time Fourier Transform (STFT)
    D = librosa.stft(y)
    
    # Convert to a magnitude spectrogram
    S = np.abs(D)

    # Get the top N wavelet coefficients
    wavelets = np.sort(S, axis=None)[-top_n:]
    mask = np.zeros_like(S, dtype=bool)
    for wavelet in wavelets:
        indices = np.where(S == wavelet)
        mask[indices] = True
    top_wavelets = np.where(mask, S, 0)

    # Plot the top wavelet spectrogram
    plt.figure(figsize=(8, 4))
    librosa.display.specshow(librosa.amplitude_to_db(top_wavelets, ref=np.max), sr=sr, y_axis='log', x_axis='time')
    plt.title(f'Top {top_n} Wavelets Spectrogram')
    plt.colorbar(format='%+2.0f dB')
    plt.tight_layout()

    # Save the spectrogram image
    plt.savefig(output_path)
    plt.close()



In [7]:
# CNN to generate embeddings of the spectrograms

In [5]:
# To keep track of unique original paths processed
processed_original_paths = set()

# Process and save spectrograms with exact directory structure and timing
output_base = 'spectrograms'

start_processing = False  # Flag to indicate when to start processing

for idx, row in audio_df.iterrows():
    song_name = row['Song Name']
    original_path = row['Original Audio Path']
    hummed_path = row['Hummed Audio Path']

    # Check if we've reached the song "00026"
    if song_name == "00046":
        start_processing = True  # Start processing from here

    if not start_processing:
        continue  # Skip all files until we reach "00026"

    # Check if the original path has already been processed
    if original_path in processed_original_paths:
        print(f"Skipping already processed original path: {original_path}")
    else:
        # Define structured output directories based on relative paths
        original_relative_path = os.path.relpath(original_path, start='MIR-QBSH/MIR-QBSH/waveFile')

        # Extract the base filename (e.g., 00026) without extensions
        original_filename = os.path.splitext(os.path.basename(original_path))[0]

        # Set up the output directory based on original paths
        original_output_dir = os.path.join(output_base, 'original', os.path.dirname(original_relative_path))
        
        os.makedirs(original_output_dir, exist_ok=True)

        # Track and print the time taken for original spectrogram
        start_time = time.time()
        audio_to_spectrogram(original_path, os.path.join(original_output_dir, f'{original_filename}.png'))
        original_duration = time.time() - start_time
        print(f"Original spectrogram for {original_filename} took {original_duration:.2f} seconds")

        # Add original path to the set of processed paths
        processed_original_paths.add(original_path)

    # Always process hummed audio
    hummed_relative_path = os.path.relpath(hummed_path, start='MIR-QBSH/MIR-QBSH/waveFile')
    hummed_filename = os.path.splitext(os.path.basename(hummed_path))[0]
    hummed_output_dir = os.path.join(output_base, 'hummed', os.path.dirname(hummed_relative_path))
    os.makedirs(hummed_output_dir, exist_ok=True)

    # Track and print the time taken for hummed spectrogram
    start_time = time.time()
    audio_to_spectrogram(hummed_path, os.path.join(hummed_output_dir, f'{hummed_filename}.png'))
    hummed_duration = time.time() - start_time
    print(f"Hummed spectrogram for {hummed_filename} took {hummed_duration:.2f} seconds")

print("All spectrogram images saved successfully.")


Original spectrogram for 00046 took 82.85 seconds
Hummed spectrogram for 00046 took 1.39 seconds
Skipping already processed original path: MIR-QBSH/MIR-QBSH/miditowav\00046.wav
Hummed spectrogram for 00046 took 1.43 seconds
Skipping already processed original path: MIR-QBSH/MIR-QBSH/miditowav\00046.wav
Hummed spectrogram for 00046 took 1.40 seconds
Skipping already processed original path: MIR-QBSH/MIR-QBSH/miditowav\00046.wav
Hummed spectrogram for 00046 took 1.58 seconds
Skipping already processed original path: MIR-QBSH/MIR-QBSH/miditowav\00046.wav
Hummed spectrogram for 00046 took 1.34 seconds
Skipping already processed original path: MIR-QBSH/MIR-QBSH/miditowav\00046.wav
Hummed spectrogram for 00046 took 1.46 seconds
Skipping already processed original path: MIR-QBSH/MIR-QBSH/miditowav\00046.wav
Hummed spectrogram for 00046 took 1.33 seconds
Skipping already processed original path: MIR-QBSH/MIR-QBSH/miditowav\00046.wav
Hummed spectrogram for 00046 took 1.41 seconds
Skipping alrea

Hummed spectrogram for 00046 took 1.41 seconds
Skipping already processed original path: MIR-QBSH/MIR-QBSH/miditowav\00046.wav
Hummed spectrogram for 00046 took 1.31 seconds
Skipping already processed original path: MIR-QBSH/MIR-QBSH/miditowav\00046.wav
Hummed spectrogram for 00046 took 1.38 seconds
Skipping already processed original path: MIR-QBSH/MIR-QBSH/miditowav\00046.wav
Hummed spectrogram for 00046 took 1.31 seconds
Skipping already processed original path: MIR-QBSH/MIR-QBSH/miditowav\00046.wav
Hummed spectrogram for 00046 took 1.40 seconds
Skipping already processed original path: MIR-QBSH/MIR-QBSH/miditowav\00046.wav
Hummed spectrogram for 00046 took 1.31 seconds
Skipping already processed original path: MIR-QBSH/MIR-QBSH/miditowav\00046.wav
Hummed spectrogram for 00046 took 1.36 seconds
Skipping already processed original path: MIR-QBSH/MIR-QBSH/miditowav\00046.wav
Hummed spectrogram for 00046 took 1.29 seconds
Skipping already processed original path: MIR-QBSH/MIR-QBSH/midi

Hummed spectrogram for 00048 took 1.53 seconds
Skipping already processed original path: MIR-QBSH/MIR-QBSH/miditowav\00048.wav
Hummed spectrogram for 00048 took 1.34 seconds
Skipping already processed original path: MIR-QBSH/MIR-QBSH/miditowav\00048.wav
Hummed spectrogram for 00048 took 1.52 seconds
Skipping already processed original path: MIR-QBSH/MIR-QBSH/miditowav\00048.wav
Hummed spectrogram for 00048 took 1.38 seconds
Skipping already processed original path: MIR-QBSH/MIR-QBSH/miditowav\00048.wav
Hummed spectrogram for 00048 took 1.43 seconds
Skipping already processed original path: MIR-QBSH/MIR-QBSH/miditowav\00048.wav
Hummed spectrogram for 00048 took 1.47 seconds
Skipping already processed original path: MIR-QBSH/MIR-QBSH/miditowav\00048.wav
Hummed spectrogram for 00048 took 1.43 seconds
Skipping already processed original path: MIR-QBSH/MIR-QBSH/miditowav\00048.wav
Hummed spectrogram for 00048 took 1.45 seconds
Skipping already processed original path: MIR-QBSH/MIR-QBSH/midi

In [19]:
import os
import pandas as pd

# Directories containing the original and hummed spectrogram files
original_dir = 'spectrograms/miditowav'
hummed_root_dir = 'spectrograms/hummed'

# Collect all original spectrogram files (assumes one original per song)
original_files = [os.path.join(original_dir, f).replace('\\', '/') for f in os.listdir(original_dir) if f.endswith('.png')]

# Prepare lists to store DataFrame columns
song_names = []
original_paths = []
hummed_paths = []

# Loop through each original spectrogram file and find corresponding hummed spectrograms
for original_path in original_files:
    song_name = os.path.splitext(os.path.basename(original_path))[0]  # Extract song name (without extension)

    # Find corresponding hummed spectrograms in hummed_root_dir by traversing subdirectories
    for year in os.listdir(hummed_root_dir):
        year_path = os.path.join(hummed_root_dir, year)
        if os.path.isdir(year_path):  # Ensure it's a directory
            for person in os.listdir(year_path):
                person_path = os.path.join(year_path, person)
                if os.path.isdir(person_path):  # Ensure it's a directory
                    for hummed_file in os.listdir(person_path):
                        if hummed_file.endswith('.png'):
                            hummed_song_name = os.path.splitext(hummed_file)[0]  # Extract song name from hummed file
                            if hummed_song_name == song_name:  # Match song name
                                hummed_path = os.path.join(person_path, hummed_file).replace('\\', '/')
                                
                                # Append data for each hummed version
                                song_names.append(song_name)
                                original_paths.append(original_path)  # Duplicate original path
                                hummed_paths.append(hummed_path)

# Create the DataFrame
data = {
    'Song Name': song_names,
    'Original Spectrogram Path': original_paths,
    'Hummed Spectrogram Path': hummed_paths
}
spectrogram_df = pd.DataFrame(data)

print(spectrogram_df.head())  # Display the first few rows to verify


  Song Name         Original Spectrogram Path  \
0     00001  spectrograms/miditowav/00001.png   
1     00001  spectrograms/miditowav/00001.png   
2     00001  spectrograms/miditowav/00001.png   
3     00001  spectrograms/miditowav/00001.png   
4     00001  spectrograms/miditowav/00001.png   

                             Hummed Spectrogram Path  
0  spectrograms/hummed/year2006a/person00001/0000...  
1  spectrograms/hummed/year2006a/person00002/0000...  
2  spectrograms/hummed/year2006a/person00003/0000...  
3  spectrograms/hummed/year2006a/person00004/0000...  
4  spectrograms/hummed/year2006a/person00005/0000...  


In [20]:
audio_df.to_csv('spectrogram_df.csv', index=False)